In [41]:
from datetime import datetime
from datetime import timedelta
from sportsreference.ncaab.boxscore import Boxscores
from sportsreference.ncaab.teams import Teams
import pandas as pd
import csv
import os
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait # available since 2.4.0
from selenium.webdriver.support import expected_conditions as EC # available since 2.26.0
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from csv import reader
from csv import writer

In [42]:
def get_today():
    today = datetime.today()
    year = today.year
    month = today.month
    day = today.day
    return year,month,day

def get_yesterday():
    today = datetime.today()
    yesterday = today - timedelta(days = 1)
    year = yesterday.year
    month = yesterday.month
    day = yesterday.day
    return year,month,day

def get_games(year, month ,day):
    home_teams = []
    away_teams = []
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        
    name = str(year)+"_"+str(month)+"_"+str(day)+"_games.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home_team","away_team"])
        for i,team in enumerate(home_teams):
            writer.writerow([home_teams[i],away_teams[i]])
            
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"games\\"+name)
    
    return home_teams,away_teams


def get_scores(year, month ,day):
    home_teams = []
    away_teams = []
    home_scores = []
    away_scores = []
    
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        home_scores.append(game['home_score'])
        away_scores.append(game['away_score'])
    
    name = str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home_team","away_team","home_score","away_score"])
        for i,team in enumerate(home_teams):
            writer.writerow([home_teams[i],away_teams[i],home_scores[i],away_scores[i]])
            
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"scores\\"+name)       
    return home_teams,away_teams,home_scores,away_scores

def kenpom(year, month ,day):
    options = webdriver.ChromeOptions() 
    #options.add_argument(r'''user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data''')
    #"user-data-dir=C:\Users\Tarik's PC\AppData\Local\Google\Chrome\User Data" home
    # "user-data-dir=C:\Users\koric1\AppData\Local\Google\Chrome\User Data" work
    #options.add_argument("--start-maximized")
    #self.options.add_argument("--headless")
    #self.options.add_argument("--no-sandbox")
    #self.options.add_argument("--disable-gpu")
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')  # Last I checked this was necessa
    driver = webdriver.Chrome(chrome_options= options)


    name = str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"

    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["team","off","def","tempo","luck"])
        #writer.writerow(["SN", "Name", "Contribution"])
        driver.get('https://kenpom.com/')
        time.sleep(1)

        for x in range(1,10):
            try:
                for z in range(1,41):

                    team_name = str(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[2]/a').text)
                    team_off = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[6]').text)
                    team_def = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[8]').text)
                    team_t = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[10]').text)
                    team_luck = float(driver.find_element_by_xpath('//*[@id="ratings-table"]/tbody['+str(x)+']/tr['+str(z)+']/td[12]').text)
                    #print(team_name,team_off,team_def,team_t,team_luck)
                    writer.writerow([team_name,team_off,team_def,team_t,team_luck])
            except:
                break
    path = os.getcwd()
    os.rename(path+"\\"+name, path+"\\"+"kenpom\\"+name)
    driver.close()
    

In [95]:
#combine for model
def score_kenmpom(year,month,day):
    path = os.getcwd() 
    scores = path+"\\scores\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores.csv"
    kenpom =  path+"\\kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"
    name = path+"\\scores_kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores_kenpom.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home","away","home_score","away_score","home_off","home_def","home_tempo","home_luck","away_off","away_def","away_tempo","away_luck"])
        with open(scores, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    home_team = row[0]
                    away_team = row[1]
                    home_score = row[2]
                    away_score = row[3]


                    

                    

                    if home_team in map_names_not_same.keys():
                        home_team = map_names_not_same[home_team]
                    else:
                        name_list = home_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            home_team = new_name
                        
                    
                    if away_team in map_names_not_same.keys():
                        away_team = map_names_not_same[away_team]
                    else:
                        name_list = away_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            away_team = new_name
                    
                    print(home_team, away_team)

                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == home_team:
                                    home_off = row2[1]
                                    home_def = row2[2]
                                    home_tempo = row2[3]
                                    home_luck = row2[4]
                                    break
                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == away_team:
                                    away_off = row2[1]
                                    away_def = row2[2]
                                    away_tempo = row2[3]
                                    away_luck = row2[4]
                                    break
                    
                    writer.writerow([home_team,away_team,home_score,away_score,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck])
      
    

In [96]:
#add to all 
def add_to_all(year,month,day):
    path = os.getcwd() 
    name = path+"\\scores_kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_scores_kenpom.csv"
    date = str(year)+"-"+str(month)+"-"+str(day)
    with open("all.csv", 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        with open(name, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    csv_writer.writerow([row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],row[11],date])

In [100]:
def predict(year,month,day):
    path = os.getcwd() 
    games = path+"\\games\\"+str(year)+"_"+str(month)+"_"+str(day)+"_games.csv"
    name = path+"\\predict\\"+str(year)+"_"+str(month)+"_"+str(day)+"_predict.csv"
    kenpom = path+"\\kenpom\\"+str(year)+"_"+str(month)+"_"+str(day)+"_kenpom.csv"
    with open(name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["home","away","home_off","home_def","home_tempo","home_luck","away_off","away_def","away_tempo","away_luck"])
        with open(games, 'r') as read_obj:
            csv_reader = reader(read_obj)
            header = next(csv_reader)
            if header != None:
                for row in csv_reader:
                    home_team = row[0]
                    away_team = row[1]


                

                    if home_team in map_names_not_same.keys():
                        home_team = map_names_not_same[home_team]
                    else:
                        name_list = home_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            home_team = new_name
                        
                    
                    if away_team in map_names_not_same.keys():
                        away_team = map_names_not_same[away_team]
                    else:
                        name_list = away_team.split()
                        if name_list[-1] == "State":
                            name_list[-1] = "St."
                            new_name = " ".join(str(x) for x in name_list)
                            away_team = new_name


                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == home_team:
                                    home_off = row2[1]
                                    home_def = row2[2]
                                    home_tempo = row2[3]
                                    home_luck = row2[4]
                                    break
                    with open(kenpom, 'r') as read_obj2:
                        csv_reader2 = reader(read_obj2)
                        header2 = next(csv_reader2)
                        if header2 != None:
                            for row2 in csv_reader2:
                                if row2[0] == away_team:
                                    away_off = row2[1]
                                    away_def = row2[2]
                                    away_tempo = row2[3]
                                    away_luck = row2[4]
                                    break
                    
                    writer.writerow([home_team,away_team,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck])

In [101]:
score_kenmpom(2020,2,19)

N.C. State Duke
Louisville Syracuse
DePaul Villanova
Georgia Auburn
Seton Hall Butler
Houston Tulsa
Houston Baptist Abilene Christian
Fresno St. Air Force
Loyola Marymount Army
San Jose St. Boise St.
Virginia Boston College
Lafayette Boston University
Washington St. California
UMKC Cal Baptist
Stephen F. Austin Central Arkansas
Cincinnati UCF
The Citadel Chattanooga
Memphis East Carolina
La Salle Fordham
ETSU Furman
Richmond George Mason
Duquesne George Washington
Wake Forest Georgia Tech
Loyola Chicago Illinois St.
Minnesota Indiana
Siena Iona
Texas Tech Kansas St.
Sam Houston St. Lamar
Colgate Lehigh
UC Irvine Long Beach St.
Arkansas St. Louisiana Monroe
Incarnate Word McNeese St.
Virginia Tech Miami FL
Rutgers Michigan
Bradley Missouri St.
American Navy
Northwestern St. Nicholls St.
South Dakota North Dakota St.
Georgetown Providence
Mercer Samford
Mississippi St. South Carolina
North Dakota South Dakota St.
Tulane SMU
Alabama Texas A&M
Texas TCU
Drake Valparaiso
Western Carolina VM

In [80]:
predict(2020, 2,20)

In [65]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import AdaBoostRegressor

In [60]:
df = pd.read_csv('all.csv')
df.head()

,home,away,home_score,away_score,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck,date
0,Illinois,Maryland,66,75,110.9,93.7,66.0,0.022,113.5,92.4,67.0,0.037,2/7/2020
1,Rider,Canisius,61,60,100.9,102.1,71.1,0.036,97.7,105.7,70.0,-0.040,2/7/2020
2,Buffalo,Central Michigan,65,60,104.1,102.3,73.9,0.045,102.0,106.4,73.6,-0.036,2/7/2020
3,Brown,Dartmouth,67,65,96.4,101.1,69.4,0.120,96.7,99.9,65.8,-0.081,2/7/2020
4,VCU,Davidson,73,62,104.7,94.4,69.0,-0.064,112.5,101.9,64.8,-0.094,2/7/2020


In [62]:
X = df[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
y = df[['home_score','away_score']].values

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#linear regression
model_lin = LinearRegression()  
model_lin.fit(X_train, y_train)
y_pred = model_lin.predict(X_test)
model_lin_mae = mean_absolute_error(y_test, y_pred)

#random forrest
model_rf = RandomForestRegressor()
model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
model_rf_mae = mean_absolute_error(y_test, y_pred)

#ridge
model_ridge = Ridge(alpha=1.0)
model_ridge.fit(X_train, y_train)
y_pred = model_ridge.predict(X_test)
model_ridge_mae = mean_absolute_error(y_test, y_pred)


print(model_lin_mae,model_rf_mae,model_ridge_mae)

6.841096916402099 8.153153153153154 6.813988273407501


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [86]:
df2 = pd.read_csv('predict\\2020_2_20_predict.csv')
df2

,home,away,home_off,home_def,home_tempo,home_luck,away_off,away_def,away_tempo,away_luck
0,Gonzaga,San Francisco,121.3,94.4,71.9,0.050,108.3,98.2,68.7,-0.006
1,Arizona St.,Oregon,106.0,94.7,73.2,0.069,117.1,97.1,65.0,0.023
2,Colorado,USC,109.1,93.4,68.2,-0.010,104.3,92.0,68.9,0.083
3,Iowa,Ohio St.,117.3,98.6,70.2,0.005,114.3,92.0,66.2,-0.038
4,BYU,Santa Clara,116.3,95.6,69.5,-0.028,102.6,101.3,71.6,0.049
5,Arizona,Oregon St.,111.0,91.4,69.3,-0.077,110.9,102.0,65.6,-0.002
6,SIU-Edwardsville,Austin Peay,111.0,91.4,69.3,-0.077,106.9,107.0,67.8,0.032
7,Morehead St.,Belmont,94.6,108.1,67.3,-0.001,106.5,100.5,70.5,0.012
8,Utah Valley,Cal St. Bakersfield,96.5,104.2,70.4,-0.092,96.2,104.9,64.9,-0.063
9,Cal Poly,UC Davis,95.2,108.5,68.3,-0.038,104.7,109.4,68.0,-0.024


In [94]:
X = df2[['home_off','home_def','home_tempo','home_luck','away_off','away_def','away_tempo','away_luck']].values
y_pred = model_lin.predict(X).round()
y_pred

array([[87., 71.],
       [70., 73.],
       [70., 64.],
       [75., 74.],
       [87., 69.],
       [77., 65.],
       [83., 65.],
       [65., 76.],
       [70., 65.],
       [72., 74.],
       [73., 72.],
       [66., 65.],
       [74., 59.],
       [77., 74.],
       [69., 58.],
       [77., 78.],
       [75., 67.],
       [78., 73.],
       [79., 72.],
       [66., 63.],
       [63., 59.],
       [67., 74.],
       [79., 60.],
       [70., 66.],
       [70., 67.],
       [73., 56.],
       [67., 65.],
       [70., 73.],
       [68., 81.],
       [78., 71.],
       [69., 72.],
       [70., 70.],
       [80., 69.],
       [75., 66.],
       [73., 71.],
       [66., 69.],
       [66., 70.],
       [57., 66.],
       [74., 80.],
       [70., 76.],
       [71., 69.],
       [71., 63.],
       [66., 64.],
       [69., 57.],
       [76., 64.],
       [75., 64.],
       [71., 67.],
       [64., 61.],
       [61., 63.],
       [74., 71.],
       [83., 72.],
       [66., 67.],
       [64.,

In [39]:
add_to_all(2020,2,7)

In [32]:

score_kenmpom(2020,2,7)


In [38]:
year,month,day = get_yesterday()
print(year, month, day)

2020 9 22


In [36]:
type(year)

int

In [37]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 2,7)

In [8]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i])

NameError: name 'home_teams' is not defined

In [2]:
kp = pd.read_csv('kenpom.csv')
kp.head()

,team,off,def,tempo,luck
0,Abilene Christian,99.1,100.2,68.4,-0.023
1,Air Force,107.8,110.3,68.8,-0.059
2,Akron,108.5,98.9,69.4,0.022
3,Alabama,111.0,99.5,74.8,-0.076
4,Alabama A&M,87.9,108.2,67.7,0.014


In [3]:
for index, row in kp.iterrows():
    name = row['team']
    name_list = name.split()
    if name_list[-1] == "St.":
        name_list[-1] = "State"
        new_name = " ".join(str(x) for x in name_list)
        kp['team'][index] = new_name
        
        
    

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [4]:
teams_ken_pom = kp['team'].tolist()

In [5]:
teams = Teams()
teams_sportsreference = []
for team in teams:
    teams_sportsreference.append(team.name)

In [11]:
for team in teams_sportsreference:
    if team not in teams_ken_pom:
        if team not in map_names_not_same.keys():
            print(team)

In [99]:
map_names_not_same = {
                        "Alabama-Birmingham" : "UAB",
                        "Albany (NY)": "Albany",
                        "Arkansas-Pine Bluff" : "Arkansas Pine Bluff",
                        "Bethune-Cookman" : "Bethune Cookman",
                        "Bowling Green State" : "Bowling Green",
                        "Brigham Young" : "BYU",
                        "Cal State Bakersfield" : "Cal St. Bakersfield",
                        "Cal State Fullerton" : "Cal St. Fullerton",
                        "Cal State Northridge" : "Cal St. Northridge",
                        "California Baptist" : "Cal Baptist",
                        "UC-Davis" : "UC Davis",
                        "UC-Irvine" : "UC Irvine",
                        "UC-Riverside" : "UC Riverside",
                        "UC-Santa Barbara" : "UC Santa Barbara",
                        "University of California" : "California",
                        "Central Connecticut State" : "Central Connecticut",
                        "Central Florida" : "UCF",
                        "Citadel" : "The Citadel",
                        "College of Charleston" : "Charleston",
                        "Detroit Mercy" : "Detroit",
                        "Florida International" : "FIU",
                        "Gardner-Webb" : "Gardner Webb",
                        "Grambling" : "Grambling St.",
                        "Illinois-Chicago" : "Illinois Chicago",
                        "Purdue-Fort Wayne" : "Purdue Fort Wayne",
                        "Cal State Long Beach" : "Long Beach St.",
                        "Long Island University" : "LIU",
                        "Louisiana-Monroe" : "Louisiana Monroe",
                        "Louisiana State" : "LSU",
                        "Loyola (IL)" : "Loyola Chicago",
                        "Loyola (MD)" : "Loyola Marymount",
                        "Maryland-Baltimore County" : "UMBC",
                        "Maryland-Eastern Shore" : "Maryland Eastern Shore",
                        "Massachusetts-Lowell" : "UMass Lowell",
                        "Miami (FL)" : "Miami FL",
                        "Miami (OH)" : "Miami OH",
                        "Missouri-Kansas City" : "UMKC",
                        "Omaha" : "Nebraska Omaha",
                        "Nevada-Las Vegas" : "UNLV",
                        "North Carolina-Asheville" : "UNC Asheville",
                        "North Carolina-Greensboro" : "UNC Greensboro",
                        "North Carolina-Wilmington" : "UNC Wilmington",
                        "NC State" : "N.C. State",
                        "Pennsylvania" : "Penn",
                        "Prairie View" : "Prairie View A&M",
                        "Saint Francis (PA)" : "St. Francis PA",
                        "Saint Mary's (CA)" : "Saint Mary's",
                        "South Carolina Upstate" : "South Carolina St.",
                        "Southern California" : "USC",
                        "Southern Methodist" : "SMU",
                        "Southern Mississippi" : "Southern Miss",
                        "St. Francis (NY)" : "St. Francis NY",
                        "St. John's (NY)" : "St. John's",
                        "Tennessee-Martin" : "Tennessee Martin",
                        "Texas A&M-Corpus Christi" : "Texas A&M Corpus Chris",
                        "Texas-Arlington" : "UT Arlington",
                        "Texas Christian" : "TCU",
                        "Texas-El Paso" : "UTEP",
                        "Texas-Rio Grande Valley" : "UT Rio Grande Valley",
                        "Texas-San Antonio" : "UTSA",
                        "Virginia Commonwealth" : "VCU"
                     }

In [88]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 3 ,8)

In [89]:
h_off = []
h_def = []
h_tempo = []
h_luck = []
a_off = []
a_def = []
a_tempo = []
a_luck = []
not_in_h = []
not_in_a = []
for i,team in enumerate(home_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team:
            h_off.append(row['off'])
            h_def.append(row['def'])
            h_tempo.append(row['tempo'])
            h_luck.append(row['luck'])
            check = True
    if not check:
        h_off.append(None)
        h_def.append(None)
        h_tempo.append(None)
        h_luck.append(None)
        not_in_h.append(i)

for i,team in enumerate(away_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team and i not in not_in_h:
            a_off.append(row['off'])
            a_def.append(row['def'])
            a_tempo.append(row['tempo'])
            a_luck.append(row['luck'])
            check = True
    if not check:
        not_in_a.append(i)
x = 0
for i in not_in_a:
    num = i - x
    del h_off[num]
    del h_def[num]
    del h_tempo[num]
    del h_luck[num]
    del away_teams[num]
    del home_teams[num]
    x += 1
    

In [90]:
len(h_off)

17

In [91]:
len(home_teams)

17

In [93]:
for i, val in enumerate(home_teams):
    print(home_teams[i],away_teams[i], h_off[i], a_off[i], i + 1)

Maryland Michigan 113.5 113.2 1
Michigan State Ohio State 115.2 114.3 2
Illinois Iowa 110.9 117.3 3
Houston Memphis 112.7 100.6 4
Boston University Bucknell 103.3 96.2 5
Hofstra Drexel 108.7 100.0 6
UCF East Carolina 103.5 99.6 7
William & Mary Elon 103.3 100.6 8
Winthrop Hampton 104.7 104.8 9
Colgate Lafayette 107.2 100.6 10
Liberty Lipscomb 106.1 101.2 11
Minnesota Nebraska 112.1 102.0 12
South Dakota North Dakota 108.6 102.2 13
Towson Northeastern 105.7 105.4 14
Wichita State Tulsa 105.6 103.2 15
Bradley Valparaiso 106.8 104.5 16
Chattanooga Wofford 106.6 104.5 17


In [64]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i], i)

83 Maryland Michigan 70 0
80 Michigan State Ohio State 69 1
78 Illinois Iowa 76 2
64 Houston Memphis 57 3
64 Boston University Bucknell 61 4
76 Tulane UConn 80 5
67 College of Charleston Delaware 79 6
61 Hofstra Drexel 43 7
94 UCF East Carolina 62 8
63 William & Mary Elon 68 9
76 Winthrop Hampton 68 10
89 Colgate Lafayette 64 11
73 Liberty Lipscomb 57 12
107 Minnesota Nebraska 75 13
79 Oral Roberts Omaha 52 14
71 South Dakota North Dakota 74 15
62 Towson Northeastern 72 16
79 Wichita State Tulsa 57 17
80 Bradley Valparaiso 66 18
97 ETSU Western Carolina 75 19
70 Chattanooga Wofford 72 20
